In [1]:
#importing library
import pandas as pd
import scipy.stats as stats
import pylab
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pylab
import numpy as np

#Importing the various accuracy measures
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from pandas import read_csv
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor

In [2]:
# load datasets
train = pd.read_csv('train_kdhs.csv')

Define X and y values

In [3]:
maskE= train["Year"] == 2015
data_2015=train[maskE]
data_2015.head()

In [ ]:
X_test = data_2015[['Enhanced_Vegetation_Index', 'Global_Human_Footprint','ITN_Coverage','Mean_Temperature','Proximity_to_Water', 'Rainfall', 'DEM', 'Year']]
y_test = data_2015[['Malaria_Prevalence']]

In [ ]:
maskF= train["Year"] < 2015
data00_10=train[maskF]

In [ ]:
X_train = data00_10[['Enhanced_Vegetation_Index', 'Global_Human_Footprint','ITN_Coverage','Mean_Temperature','Proximity_to_Water', 'Rainfall', 'DEM', 'Year']]
y_train = data00_10[['Malaria_Prevalence']]
y_train

In [7]:
X_train.shape, y_train.shape

((972, 7), (972,))

### Second Model

In [27]:
#Model 2
RegModel=XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=500, objective='reg:linear', booster='gbtree')

In [28]:
#Printing all the parameters of XGBoost
print(RegModel)

XGBRegressor(base_score=None, booster='gbtree', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', random_state=None, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)


In [30]:
#Creating the model on Training Data
XGB=RegModel.fit(X_train,y_train)
prediction=XGB.predict(X_test)

[00:27:11] WARNING: c:\ci\xgboost-split_1619728435298\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [32]:
#Measuring Goodness of fit in Training data
from sklearn import metrics
print('R2 Value in training:',metrics.r2_score(y_train, XGB.predict(X_train)))

R2 Value in training: 0.9824363908033461


In [33]:
#Measuring accuracy on Testing Data
print('Accuracy',100- (np.mean(np.abs((y_test - prediction) / y_test)) * 100))

Accuracy 81.09543766987139


In [34]:
#Calculating the various accuracy measures
mae = metrics.mean_absolute_error(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, prediction)

In [35]:
mape = np.mean(np.abs((y_test - prediction)/y_test))*100

In [36]:
#printing the accuracy measures, rounded to 4 dp
print("Results of sklearn.metrics:")
mae = round(mae, 4)
mse = round(mse, 4)
rmse = round(rmse, 4)
r2 = round(r2*100, 4)
mape = round(mape, 4)


print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)
print("MAPE:", mape)

Results of sklearn.metrics:
MAE: 0.0187
MSE: 0.0007
RMSE: 0.0262
R-Squared: 88.6631
MAPE: 18.9046


In [37]:
#Reading the independent variables prepped from ArcMap
Ind_Vars15=pd.read_csv("New_Ind_Var15.csv")

In [38]:
#Checking if there are null values in the independent variables
Ind_Vars15.isnull().sum()

FID                       0
OBJECTID                  0
pointid                   0
LAT                       0
LONG                      0
Rainfall_15               0
LST_15                    0
Water_Bodies_Surface      0
ITN_Surface15             0
GPW_Pop15                 0
Global_Human_Footprint    0
DEM_Surface               0
dtype: int64

In [44]:
#Reading the independent var. points, and coordinates. Converting them to array
Predictors_grid = Ind_Vars15[['Rainfall','Mean_Temp','Water_Bodies_Surface','ITN_Surface','GPW_Pop','Global_Human_Footprint','DEM_Surface']]
Predictors_grid.columns=['Rainfall_2015','Land_Surface_Temperature_2015','Proximity_to_Water', 'ITN_Coverage_2015','UN_Population_Density_2015','Global_Human_Footprint', 'DEM' ]
Predictors_grid.columns

Index(['Rainfall_2015', 'Land_Surface_Temperature_2015', 'Proximity_to_Water',
       'ITN_Coverage_2015', 'UN_Population_Density_2015',
       'Global_Human_Footprint', 'DEM'],
      dtype='object')

In [45]:
grid_coordinates=Ind_Vars15[['LONG','LAT']]

In [46]:
#### Run XGB model on the predictor raster values
XGB_predictors= XGB.predict(Predictors_grid)
##fn=rf_predictors.reshape(val1.shape[1],val1.shape[2])
predi_df=pd.DataFrame(XGB_predictors)
final_gpd=pd.concat([grid_coordinates,predi_df],axis=1)
final_gpd.columns=['LONG','LAT','Predi_15']
##print(final_gpd)

In [47]:
final_gpd.to_csv("XGBPredicted_Malaria_Prevalence.csv")